In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip "/content/drive/MyDrive/data.zip" -d /content/

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/Lion/img/aa02ad2f61d2d6c4ebc87e96d4b4e53d.jpg  
  inflating: /content/__MACOSX/data/Lion/img/._aa02ad2f61d2d6c4ebc87e96d4b4e53d.jpg  
  inflating: /content/data/Lion/img/9b523a44bf6b6a9c7cd28492a3f48381.jpg  
  inflating: /content/__MACOSX/data/Lion/img/._9b523a44bf6b6a9c7cd28492a3f48381.jpg  
  inflating: /content/data/Lion/img/16dcbcaf8fb82830006cbf23e6559c60.jpg  
  inflating: /content/__MACOSX/data/Lion/img/._16dcbcaf8fb82830006cbf23e6559c60.jpg  
  inflating: /content/data/Lion/img/5273e1ba93fe59dc42686e85f397d1bc.jpg  
  inflating: /content/__MACOSX/data/Lion/img/._5273e1ba93fe59dc42686e85f397d1bc.jpg  
  inflating: /content/data/Lion/img/60ec4cac62bc459bf93fac0b78a68614.jpg  
  inflating: /content/__MACOSX/data/Lion/img/._60ec4cac62bc459bf93fac0b78a68614.jpg  
  inflating: /content/data/Lion/img/ae701cfab0d5faaebc9c4c9ed8934c38.jpg  
  inflating: /content/__MACOSX/data/Lion/img/._ae701cfab0d5faaebc9c4c

In [3]:
!pip install mwparserfromhell

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.3/196.3 kB 14.8 MB/s eta 0:00:00


In [4]:
import mwparserfromhell as mwp
import re
import json
import os
import html
import string
import shutil

from pathlib import Path
from os import listdir, mkdir
from os.path import isfile, isdir, join, exists, abspath

In [5]:
dataset_root = '/content/data'

In [6]:
def _globalMaxPool1D(tensor):
    _,_,_,size = tensor.shape
    return [tensor[:,:,:,i].max() for i in range(size)]

def _getImageFeatures(model, img_path):
    img = image.load_img(img_path, target_size=None)

    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)

    feature_tensor = model.predict(img_data)
    get_img_id = lambda p: p.split('/')[-1].split('.')[0]
    return {
        "id": get_img_id(img_path),
        "features": _globalMaxPool1D(feature_tensor),
    }

def _getJSON(path):
    with open(path) as json_file:
        return json.loads(json.load(json_file))

def _clean_text(text):
    text = text.replace("\n", " ")
    # onyshchak: only checking first 1000 characters, will need to extract summary propely
    text = text[:1000].rsplit(' ', 1)[0]
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

def _getTextFeatures(text_path):
    data = _getJSON(text_path)
    return {
        'id': data['id'],
        'text': _clean_text(data['text']),
        'title': data['title']
    }
def _getImagesMeta(path):
    data = _getJSON(path)['img_meta']
    for x in data:
        x['description'] = _clean_text(x['description'])
        x['title'] = _clean_text(x['title'])
    return data

def _getValidImagePaths(article_path):
    img_path = join(article_path, 'img/')
    return [join(img_path, f) for f in listdir(img_path) if isfile(join(img_path, f)) and f[-4:].lower() == ".jpg"]

def _dump(path, data):
    with open(path, 'w', encoding='utf8') as outfile:
        json.dump(data, outfile, indent=2, ensure_ascii=False)

def GetArticleData(article_path):
    article_data = _getTextFeatures(join(article_path, 'text.json'))
    article_data["img"] = _getImagesMeta(join(article_path, 'img/', 'meta.json'))

    return article_data

def ReadArticles(data_path, pred=None, offset=0, limit=None):
    article_paths = [join(data_path, f) for f in listdir(data_path) if isdir(join(data_path, f))]
    limit = limit if limit else len(article_paths) - offset
    limit = min(limit, len(article_paths) - offset)

    articles = []
    for i in range(offset, offset + limit):
        path = article_paths[i]
        if (i - offset) % 300 == 0: print(i - offset, "articles have been read")
        article_data = GetArticleData(path)
        if pred and not pred(i, article_data): continue

        articles.append(article_data)
        if len(articles) >= limit: break  # useless?

    print(offset + limit, "articles have been read")
    return articles
def GenerateVisualFeatures(data_path, offset=0, limit=None, model=None):
    article_paths = [join(data_path, f) for f in listdir(data_path) if isdir(join(data_path, f))]
    limit = limit if limit else len(article_paths) - offset
    limit = min(limit, len(article_paths) - offset)
    model = model if model else ResNet152(weights='imagenet', include_top=False)

    for i in range(offset, offset + limit):
        path = article_paths[i]
        print(i, path)

        meta_path = join(path, 'img/', 'meta.json')
        meta_arr = _getImagesMeta(meta_path)
        for meta in meta_arr:
            if 'features' in meta: continue
            if meta['filename'][-4:].lower() != ".jpg": continue

            img_path =  join(path, 'img/', meta['filename'])
            try:
                features = _getImageFeatures(model, img_path)['features']
                meta['features'] = [str(f) for f in features]
            except Exception as e:
                print("exception", str(e))
                print(img_path)
                continue

        _dump(meta_path, json.dumps({"img_meta": meta_arr}))

In [7]:
%%time
articles = ReadArticles(dataset_root, offset=0, limit=None)
processed_titles = _getJSON("/content/drive/MyDrive/parsed_titles.json")

0 articles have been read
300 articles have been read
600 articles have been read
900 articles have been read
1200 articles have been read
1500 articles have been read
1800 articles have been read
2100 articles have been read
2400 articles have been read
2700 articles have been read
3000 articles have been read
3300 articles have been read
3600 articles have been read
3900 articles have been read
4200 articles have been read
4500 articles have been read
4800 articles have been read
5100 articles have been read
5400 articles have been read
5638 articles have been read
CPU times: user 22.9 s, sys: 3.86 s, total: 26.7 s
Wall time: 27.9 s


In [8]:
dataset_name = 'data_w2vv'
dataset_path = join('./', dataset_name)
if exists(dataset_path):
    shutil.rmtree(dataset_path)

mkdir(dataset_path)
subsets = {
    "train": {},
    "val": {},
    "test": {},
}

for k, v in subsets.items():
    v['name'] = dataset_name + k
    v['path'] = join(dataset_path, v['name'])
    mkdir(v['path'])

    v['feature_data_path'] = join(v['path'], 'FeatureData')
    if k == 'train':
        mkdir(v['feature_data_path'])
    else:
        dst = v['feature_data_path']
        os.symlink(os.path.relpath(subsets['train']['feature_data_path'], Path(dst).parent), dst)

    v["image_sets_path"] = join(v['path'], 'ImageSets')
    mkdir(v["image_sets_path"])

    v["text_data_path"] = join(v['path'], 'TextData')
    mkdir(v["text_data_path"])

In [9]:
def to_file(arr, filepath):
    with open(filepath, 'w') as f:
        for x in arr:
            f.write("%s\n" % x)

In [10]:
is_valid_img = lambda i: 'features' in i
has_valid_img = lambda a: len([i for i in a['img'] if is_valid_img(i)]) > 0
articles = [a for a in articles if has_valid_img(a)]
len(articles)

5419

In [11]:
list2str = lambda l: " ".join([str(x) for x in l])
get_img_id = lambda i: os.path.splitext(i['filename'])[0]  # removing file extention

img_features = set(['{} {}'.format(get_img_id(i), list2str(i['features'])) for a in articles for i in a['img'] if is_valid_img(i)])
print("len(img_features) = ", len(img_features))

raw_features_file_path = join(subsets['train']["feature_data_path"], subsets['train']['name'] + ".features.txt")
to_file(img_features, raw_features_file_path)

len(img_features) =  45643


In [12]:
def map_data(articles):
    seen = set()
    res = []
    for a in articles:
        for i in a['img']:
            if 'features' not in i: continue

            img_id = os.path.splitext(i['filename'])[0]  # removing file extention
            if img_id in seen:
                # onyshchak: if image used in 2 articles, we only take the first one for simplicity
                # TODO: use all the infomation without breaking the model
                continue

            seen.add(img_id)
            res.append({
                "filename": img_id,
                'article_id': a['id'],
                'article_title': a['title'],
                "title": os.path.splitext(i['title'])[0],
                "description": i['description'],
                "text": a['text'],
                "features": i['features'],
            })

    return res

In [13]:
from sklearn.model_selection import train_test_split

ARTICLE_LEVEL_SPLIT = True
seed = 1234

if ARTICLE_LEVEL_SPLIT:
    train, test = train_test_split(articles, test_size=0.04, random_state=seed)
    train, val = train_test_split(train, test_size=0.043, random_state=seed)

    subsets['train']['data'] = map_data(train)
    subsets['val']['data'] = map_data(val)
    subsets['test']['data'] = map_data(test)
else:
    mapped_images = map_data(articles)
    # 2325 test, 2057 val
    train, test = train_test_split(mapped_images, test_size=1000, random_state=seed)
    train, val = train_test_split(train, test_size=1000, random_state=seed)

    subsets['train']['data'] = train
    subsets['val']['data'] = val
    subsets['test']['data'] = test

subset_split_info = str([(k, len(v['data'])) for k, v in subsets.items()]).replace("(", "").replace(")", "")
subset_split_info

"['train', 42143, 'val', 2188, 'test', 1951]"

In [14]:
ids = [x['article_id'] for x in subsets['test']['data']]
to_file(ids, join(subsets['test']['path'], "test_articles_ids.txt"))

In [15]:
for v in subsets.values():
    ids = [x['filename'] for x in v['data']]
    to_file(ids, join(v["image_sets_path"], v['name'] + ".txt"))

In [16]:
%%time
# onyshchak: originally ID also contained file extention e.g. *.jpg. but not in image_sets_path
def get_description(z):
    if z['description']: return z['description']
    elif z['filename'] in processed_titles: return processed_titles[z['filename']]
    else:
        print("Missing title", z['filename'])
        return z['title']

for v in subsets.values():
    text_data = sorted(
        ['{}#enc#0 {}'.format(x['filename'], x['text']) for x in v['data']] +
        ['{}#enc#1 {}'.format(x['filename'], get_description(x)) for x in v['data']]
    )

    to_file(text_data, join(v["text_data_path"], v['name'] + ".caption.txt"))

CPU times: user 238 ms, sys: 52.1 ms, total: 290 ms
Wall time: 289 ms


In [17]:
del articles
del text_data
del train
del val
del ids
del test
del img_features
for k,v in subsets.items():
    del v['data']

In [18]:
!sudo apt install python2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython2-stdlib libpython2.7-minimal libpython2.7-stdlib python2-minimal
  python2.7 python2.7-minimal
Suggested packages:
  python2-doc python-tk python2.7-doc binfmt-support
The following NEW packages will be installed:
  libpython2-stdlib libpython2.7-minimal libpython2.7-stdlib python2
  python2-minimal python2.7 python2.7-minimal
0 upgraded, 7 newly installed, 0 to remove and 34 not upgraded.
Need to get 4,012 kB of archives.
After this operation, 16.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 libpython2.7-minimal amd64 2.7.18-13ubuntu1.5 [347 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 python2.7-minimal amd64 2.7.18-13ubuntu1.5 [1,400 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 python2-minimal amd64 2.7.18-3 [20.8 

In [19]:
!apt-get install python2

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python2 is already the newest version (2.7.18-3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [20]:
! apt install --assume-yes python-pip > /dev/null
! python2 -m pip install --user numpy scipy matplotlib ipython jupyter pandas nose > /dev/null



ERROR: Could not find a version that satisfies the requirement comm>=0.1.3 (from ipywidgets->jupyter) (from versions: 0.0.1)
ERROR: No matching distribution found for comm>=0.1.3 (from ipywidgets->jupyter)


In [21]:
!git clone https://github.com/OlehOnyshchak/w2vv.git

Cloning into 'w2vv'...
remote: Enumerating objects: 162, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 162 (delta 0), reused 2 (delta 0), pack-reused 159 (from 1)
Receiving objects: 100% (162/162), 4.85 MiB | 18.13 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [22]:
!apt-get install python2-pip -y
!pip2 install numpy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package python2-pip
  Using cached numpy-1.16.6-cp27-cp27mu-manylinux1_x86_64.whl (17.0 MB)


In [23]:
IS_FILE_LIST = 0
FEATURE_DIMENTION = 2048
feature_data_path = subsets['train']["feature_data_path"]
bin_features_path = join(feature_data_path, "pyresnet152-pool5os/")

! python2 /content/w2vv/simpleknn/txt2bin.py $FEATURE_DIMENTION $raw_features_file_path $IS_FILE_LIST $bin_features_path --overwrite 1

>>> Processing ./data_w2vv/data_w2vvtrain/FeatureData/data_w2vvtrain.features.txt
45643 lines parsed, 45643 ids,  0 failed ->  45643 unique ids


In [24]:
! mv $dataset_name/* ./
! rmdir $dataset_name

In [25]:
version_notes = "version_notes.txt"
! echo article level = $ARTICLE_LEVEL_SPLIT > $version_notes
! echo seed = $seed >> $version_notes
! echo "caption 0 = article summary" >> $version_notes
! echo "caption 1 = description(or parsed titles)" >> $version_notes
! echo split = $subset_split_info >> $version_notes

! cat $version_notes

article level = True
seed = 1234
caption 0 = article summary
caption 1 = description(or parsed titles)
split = [train, 42143, val, 2188, test, 1951]


In [26]:
! cat /content/drive/MyDrive/version_notes.txt

article level = True
seed = 1234
caption 0 = article summary
caption 1 = description(or parsed titles)
split = [train, 42140, val, 2123, test, 2118]


In [27]:
! apt-get install --assume-yes python-pip > /dev/null
! python2 -m pip install --upgrade pip --user > /dev/null
! python2 -m pip --version

pip 20.3.4 from /usr/lib/python2.7/dist-packages/pip (python 2.7)


In [28]:
! apt-get install --assume-yes python-pydot python-pydot-ng graphviz > /dev/null
! python2 -m pip install --user --no-warn-script-location numpy scipy matplotlib ipython jupyter pandas > /dev/null
! python2 -m pip install --user --no-warn-script-location tensorflow==1.15 keras tensorboard_logger pydot graphviz > /dev/null

E: Unable to locate package python-pydot
E: Unable to locate package python-pydot-ng
ERROR: Could not find a version that satisfies the requirement comm>=0.1.3 (from ipywidgets->jupyter) (from versions: 0.0.1)
ERROR: No matching distribution found for comm>=0.1.3 (from ipywidgets->jupyter)
ERROR: Operation cancelled by user


In [29]:
! nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [37]:
%%time
import shutil
shutil.copytree("/content/w2vv", "/kaggle/working/w2vv")
shutil.copytree("/kaggle/input/w2vv-wiki-model24/train_results/6e2ad6898813fe17ac064c98e0c73cf1731349e6/", "/kaggle/working/pretrained_model")
! mv /kaggle/working/w2vv/* /kaggle/working/

_ = shutil.copytree("/kaggle/input/dataset-preprocessing/", "/kaggle/working/VisualSearch")

FileExistsError: [Errno 17] File exists: '/kaggle/working/w2vv'

In [71]:
train_set_name = "data_w2vvtrain"
val_set_name = "data_w2vvval"
test_set_name = "data_w2vvtest"
feature_name = "pyresnet152-pool5os"
overwrite = 1
n_caption = 2
max_epochs = 3
BoW_size = 5
vocab_name = 'word_vocab_{}.txt'.format(BoW_size)
pretrained_model_path = '/content/best_model.h5'

In [32]:
!unzip "/content/drive/MyDrive/best_model.h5.zip" -d /content/

Archive:  /content/drive/MyDrive/best_model.h5.zip
  inflating: /content/best_model.h5  


In [33]:
!python2 -m pip install keras==2.1.5
!python2 -m pip install tensorflow==1.5.0

     |████████████████████████████████| 334 kB 3.8 MB/s 
  Using cached scipy-1.2.3-cp27-cp27mu-manylinux1_x86_64.whl (24.8 MB)
     |████████████████████████████████| 574 kB 70.0 MB/s 
  Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
     |████████████████████████████████| 44.4 MB 174 kB/s 
     |████████████████████████████████| 114 kB 81.8 MB/s 
     |████████████████████████████████| 1.0 MB 65.3 MB/s 
  Using cached enum34-1.1.10-py2-none-any.whl (11 kB)
     |████████████████████████████████| 3.0 MB 85.6 MB/s 
     |████████████████████████████████| 298 kB 54.2 MB/s 
     |████████████████████████████████| 87 kB 10.9 MB/s 
     |████████████████████████████████| 889 kB 14.8 MB/s 
Using legacy 'setup.py install' for absl-py, since package 'wheel' is not installed.
Using legacy 'setup.py install' for html5lib, since package 'wheel' is not installed.
    Running setup.py install for absl-py ... done
    Running setup.py install for html5lib ... done


In [40]:
%cd /content
! sh /content/w2vv/do_gene_vocab.sh $train_set_name $BoW_size

/content
Using TensorFlow backend.
processing data_w2vvtrain ...
0000b982dcfe78d77a9af1664607cec7#enc#0 Netley Abbey is a ruined late medieval monastery in the village of Netley near Southampton in Hampshire England The abbey was founded in 1239 as a house for monks of the austere Cistercian order Despite royal patronage Netley was never rich produced no influential scholars nor churchmen and its nearly 300year history was quiet The monks were best known to their neighbours for the generous hospitality they offered to travellers on land and sea  In 1536 Netley Abbey was seized by Henry VIII of England during the Dissolution of the Monasteries and the buildings granted to William Paulet a wealthy Tudor politician who converted them into a mansion The abbey was used as a country house until the beginning of the eighteenth century after which it was abandoned and partially demolished for building materials Subsequently the ruins became a tourist attraction and provided inspiration to poet

In [35]:
!python2 -m pip install tensorboard_logger

     |████████████████████████████████| 2.1 MB 4.7 MB/s 


In [50]:
!unzip /content/word2vec/flickr/vec500flickr30m/id.txt.zip -d /content/word2vec/flickr/vec500flickr30m/

Archive:  /content/word2vec/flickr/vec500flickr30m/id.txt.zip
  inflating: /content/word2vec/flickr/vec500flickr30m/id.txt  


In [44]:
# 确保目录存在
import os

os.makedirs("/content/word2vec/flickr/vec500flickr30m", exist_ok=True)

# 写入 shape.txt
with open("/content/word2vec/flickr/vec500flickr30m/shape.txt", "w") as f:
    f.write("1743364 500\n")

In [57]:
!unzip /content/drive/MyDrive/feature.bin.zip -d /content/word2vec/flickr/vec500flickr30m/

Archive:  /content/drive/MyDrive/feature.bin.zip
  inflating: /content/word2vec/flickr/vec500flickr30m/feature.bin  


In [59]:
!python2 -m pip install h5py

     |████████████████████████████████| 2.8 MB 4.2 MB/s 


In [72]:
! python2 /content/w2vv/w2vv_trainer.py $train_set_name $val_set_name  $test_set_name \
    --overwrite $overwrite --img_feature $feature_name --n_caption $n_caption \
    --max_epochs $max_epochs --bow_vocab $vocab_name --rnn_vocab $vocab_name

Using TensorFlow backend.
2025-04-21 00:15:04.496488: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
{
  "clipnorm": 5.0, 
  "l2_p": 0.0, 
  "simi_fun": "cosine", 
  "max_epochs": 3, 
  "img_feature": "pyresnet152-pool5os", 
  "bow_vocab": "word_vocab_5.txt", 
  "word2vec": "vec500flickr30m", 
  "n_caption": 2, 
  "overwrite": 1, 
  "text_style": "gru@bow_filterstop@word2vec_filterstop", 
  "checkpoint": "cv_w2vv_new", 
  "lr": 0.0001, 
  "corpus": "flickr", 
  "optimizer": "rmsprop", 
  "sent_maxlen": 32, 
  "init_model_from": "", 
  "dropout": 0.2, 
  "rootpath": "/root/VisualSearch", 
  "batch_size": 100, 
  "n_text_layers": "0-2048-2048", 
  "rnn_size": 1024, 
  "loss_fun": "mse", 
  "rnn_vocab": "word_vocab_5.txt", 
  "postfix": "run_0", 
  "hidden_act": "relu", 
  "model_name": "w2vv_ms"
}
/content/data_w2vvtrain/train_results/24b0805435bbb4ad467b423ac689db

In [76]:
! sh /content/do_test_data_w2vvtest.sh # shoud be do_test_${testCollection}.sh

Using TensorFlow backend.
2025-04-21 02:21:55.132697: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
{
  "weight_name": "epoch_2.h5", 
  "n_caption": 2, 
  "rootpath": "/content", 
  "overwrite": 0, 
  "model_path": "/content/data_w2vvtrain/train_results/24b0805435bbb4ad467b423ac689db73add95096"
}
gru
21/04/2025 02:21:55 INFO [/content/w2vv/util/text2vec.pyc.Index2Vec] initializing ...
21/04/2025 02:21:55 INFO [/content/w2vv/util/text2vec.pyc.BoW2VecFilterStop] initializing ...
21/04/2025 02:21:55 INFO [/content/w2vv/util/text2vec.pyc.BoW2VecFilterStop] 47159 words
21/04/2025 02:21:55 INFO [/content/w2vv/util/text2vec.pyc.AveWord2VecFilterStop] initializing ...
[BigFile] 1743364x500 instances loaded from /content/word2vec/flickr/vec500flickr30m
[BigFile] 45643x2048 instances loaded from /content/data_w2vvtest/FeatureData/pyresnet152-pool5os
21/04/2025 02:21:59 IN